In [1]:
def normalize(grid):
    """
    Given a grid of unnormalized probabilities, computes the
    correspond normalized version of that grid. 
    """
    total = 0.0
    for row in grid:
        for cell in row:
            total += cell
    for i,row in enumerate(grid):
        for j,cell in enumerate(row):
            grid[i][j] = float(cell) / total
    return grid


def blur(grid, blurring):
    """
    Spreads probability out on a grid using a 3x3 blurring window.
    The blurring parameter controls how much of a belief spills out
    into adjacent cells. If blurring is 0 this function will have 
    no effect. 
    """
    height = len(grid)
    width  = len(grid[0])

    center_prob = 1.0-blurring
    corner_prob = blurring / 12.0
    adjacent_prob = blurring / 6.0

    window = [
            [corner_prob,  adjacent_prob,  corner_prob],
            [adjacent_prob, center_prob,  adjacent_prob],
            [corner_prob,  adjacent_prob,  corner_prob]
        ]
    new = [[0.0 for i in range(width)] for j in range(height)]
    for i in range(height):
        for j in range(width):
            grid_val = grid[i][j]
            for dx in range(-1,2):
                for dy in range(-1,2):
                    mult = window[dx+1][dy+1]
                    new_i = (i + dy) % height
                    new_j = (j + dx) % width
                    new[new_i][new_j] += mult * grid_val
    return normalize(new)

In [2]:
def initialize_beliefs(grid):
    height = len(grid)
    width = len(grid[0])
    area = height * width
    belief_per_cell = 1.0 / area
    beliefs = []
    for i in range(height):
        row = []
        for j in range(width):
            row.append(belief_per_cell)
        beliefs.append(row)
    return beliefs

def sense(color, grid, beliefs, p_hit, p_miss):
    new_beliefs = []
    
    height = len(grid)
    width = len(grid[0])
    
    # loop through all grid cells
    for i in range(height):
        row = []
        for j in range(width):
            hit = (color == grid[i][j])
            row.append(beliefs[i][j] * (hit * p_hit + (1-hit) * p_miss))
        new_beliefs.append(row)
        
    s = sum(map(sum, new_beliefs))

    for i in range(height):
        for j in range(width):
            new_beliefs[i][j] = new_beliefs[i][j] / s
            
    return new_beliefs

def move(dy, dx, beliefs, blurring):
    height = len(beliefs)
    width = len(beliefs[0])
    new_G = [[0.0 for i in range(width)] for j in range(height)]
    for i, row in enumerate(beliefs):
        for j, cell in enumerate(row):
            # debugging info : the "height" and "width" in the modulo part was interchanged.
            new_i = (i + dy ) % height
            new_j = (j + dx ) % width
            new_G[int(new_i)][int(new_j)] = cell
    return blur(new_G, blurring)

In [3]:
R = 'r'
G = 'g'
grid = [
    [R,G,G,G,R],
    [G,G,R,G,R],
    [G,R,G,G,G],
    [R,R,G,R,G],
    [R,G,R,G,R],
]
blur = 0.05
p_hit = 200.0
p_miss = 1.0
colour = 'red'
dy = random.choice([-1,0,1])
dx = random.choice([-1,0,1])

In [4]:
def initialize_beliefs_c(grid):
    height = len(grid)
    width = len(grid[0])
    beliefs = [[1.0/(height*width) for j in range(width)] for i in range(height)]
    return beliefs

def sense_c(color, grid, beliefs, p_hit, p_miss):
    multiplier = lambda x: p_hit if x == color else p_miss
    single_beliefs = [[multiplier(grid[i][j]) for j in range(len(grid[0]))] for i in range(len(grid))]
    s = sum(map(sum, single_beliefs))
    new_beliefs = [[single_beliefs[i][j]/s for j in range(len(grid[0]))] for i in range(len(grid))]
    
    return new_beliefs



def move_c(dy, dx, beliefs, blurring):
    new_G = [[0.0 for i in range(len(beliefs[0]))] for j in range(len(beliefs))]
    new_i = [(i + dy ) % height for i in range(len(beliefs[0]))]
    new_j = [(j + dx ) % width for j in range(len(beliefs))]
    
    
def move_c(dy, dx, beliefs, blurring):
    height = len(beliefs)
    width = len(beliefs[0])
    new_G = [[0.0 for i in range(width)] for j in range(height)]
    for i, row in enumerate(beliefs):
        for j, cell in enumerate(row):
            # debugging info : the "height" and "width" in the modulo part was interchanged.
            new_i = (i + dy ) % height
            new_j = (j + dx ) % width
            new_G[int(new_i)][int(new_j)] = cell
    return blur(new_G, blurring)

In [5]:
beliefs_c = initialize_beliefs_c(grid)
beliefs_m = initialize_beliefs(grid)

In [6]:
beliefs_c

[[0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04]]

In [7]:
sense_c(colour, grid, beliefs_c, p_hit, p_miss)

[[0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04]]

In [8]:
initialize_beliefs(grid)

[[0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04]]

In [9]:
sense(colour, grid, beliefs_m, p_hit, p_miss)

[[0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04],
 [0.04, 0.04, 0.04, 0.04, 0.04]]

https://github.com/chpolyzo/ISDC/blob/main/bayesian_thinking/move_sense.ipynb